In [12]:
!python --version

/usr/bin/sh: 1: python: not found


In [11]:
!pip install --upgrade selenium

Defaulting to user installation because normal site-packages is not writeable


In [10]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [9]:
# Chrome 옵션 설정
options = Options()
options.binary_location = "./data/chrome-linux64/chrome" # Chrome 경로 설정
options.add_argument("--no-sandbox") # sandbox 설정
options.add_argument("--disable-dev-shm-usage") # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # 화면 크기 설정, 반응형 웹 대비
options.add_argument("--headless")  # Jupyter에서 반드시 필요한 설정
options.add_argument("--disable-gpu") # GPU 비활성화

# ChromeDriver 설정
service = Service("./data/chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.starbucks.co.kr/index.do")
print(driver.title)

# 요소 찾기
hover_element = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a')
menu_option = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a')

# 마우스 호버 액션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

# 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(5)

# 입력창 선택 후 검색
driver.find_element(By.CSS_SELECTOR,'#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,'#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(6) > a').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,'#mCSB_2_container > ul > li:nth-child(1) > a').click()
time.sleep(1)

html = driver.page_source
print(html)
driver.quit()

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [124]:
star_list = []
soup = bs(html, 'html.parser')

contents = soup.select('#mCSB_3_container > ul > li')
# print(contents[0])
# print(contents[0].find('strong').text)
# print(contents[0].find('p').text[:-9])
# print(contents[0].find('i').get('class')[0])
for i, c in enumerate(contents):
    star_list.append([c.find('strong').text.strip(), c.find('p').text[:-9].strip()])

    if c.find('i').get('class')[0] == 'pin_generalDT':
        star_list[i].append('DT')
    elif c.find('i').get('class')[0] == 'pin_general':
        star_list[i].append('GENERAL')
    elif c.find('i').get('class')[0] == 'pin_generalWT':
        star_list[i].append('WT')
    elif c.find('i').get('class')[0] == 'pin_reserve':
        star_list[i].append('RESERVE')
        
    star_list[i].append(c['data-lat'])
    star_list[i].append(c['data-long'])

for i in range(len(star_list)):
    star_list[i] = tuple(star_list[i])
    # print(len(star_list[i]))
print(star_list[22])

('서면로R', '부산광역시 부산진구 서면로 49-1 (부전동)', 'RESERVE', '35.15524851626055', '129.0576381743508')


In [151]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        host = "210.119.14.56",
        user = "root",
        password = "1234"
    )
    
    if conn.is_connected():
        print("MySQL 접속 완료")
        
except mysql.connector.Error as e:
    print("MySQL 접속 실패", e)

MySQL 접속 완료


In [152]:
db_query = "CREATE DATABASE IF NOT EXISTS JHS8144"
cursor = conn.cursor()
cursor.execute(db_query)
conn.close()
print('ok')

ok


In [153]:
conn = mysql.connector.connect(
        host = "210.119.14.56",
        user = "root",
        password = "1234",
        database = "JHS8144"
    )
tb_query = "CREATE TABLE IF NOT EXISTS JHS (STORENAME VARCHAR(100) PRIMARY KEY , LOCATION VARCHAR(200), TYPE VARCHAR(50), LATITUDE VARCHAR(100), LONGITUDE VARCHAR(100))"
cursor = conn.cursor()
cursor.execute(tb_query)
print('ok')

ok


In [157]:
insert_query = "INSERT INTO JHS (STORENAME, LOCATION, TYPE, LATITUDE, LONGITUDE) VALUES (%s, %s, %s, %s, %s)"
cursor.executemany(insert_query,star_list)
conn.commit()
print('ok')

ok


In [156]:
DELETE_QUERY = "DELETE FROM JHS"
cursor.execute(DELETE_QUERY)
print('OK')

OK


In [ ]:
SELECT_QUERY = "SELECT * FROM JHS"
RESULT = cursor.execute(SELECT_QUERY)
PRINT(RESULT)